### Statistical Regression Sept 2022 Rating
Using the DES Job Seeker file

In [ ]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

Reading in participant characteristics from the stream_participants and jobseeker table from the DB:

In [ ]:
df_ap = pd.read_csv("C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/Stat Regression/ap_stat_variables.csv") # stream_participants + jobseeker 
dis_type = pd.read_csv("C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/Stat Regression/participant_disability_type.csv") # mental/physical disability mapping
dis_type.columns = map(str.upper, dis_type.columns)

# capitalising the column names in df_ap: 
df_ap = df_ap.rename(columns={'ess_identifier' : 'JOB_SEEKER_ID'})
df_ap.columns = map(str.upper, df_ap.columns)

df_ap['JOB_SEEKER_ID'] = pd.to_numeric(df_ap['JOB_SEEKER_ID'], errors='coerce')
df_ap = df_ap[df_ap['JOB_SEEKER_ID'].isna() == False]

df_ap['DISABILITY_TYPE_CODE'] = df_ap['DISABILITY_TYPE_CODE'].astype('object')
dis_type['DISABILITY_TYPE_CODE'] = dis_type['DISABILITY_TYPE_CODE'].astype('object')
# joining the disability type to the df_ap:
df_ap = df_ap.merge(dis_type, on=['DISABILITY_TYPE_CODE'])



df = df_ap




#### Missing values

In [ ]:
naVal = df.isna().sum() * 100 / len(df)
missValDf = pd.DataFrame({'column_name' : df.columns, 'percent_missing' : naVal})
missValDf.sort_values(by=['percent_missing'], ascending=False)

Remove NA values for JOB_SEEKER_ID and AGE_AT_EXTRACT

In [ ]:
df.dtypes

Getting the Job_Seeker_Ids for the Dec 2020 star ratings - importing from the AimBig Star Prediction notebook - IDs that are eligible for 13 week outcomes:


In [ ]:
df_13 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_13_pre.csv")
df_13 = df_13.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_13['JOB_SEEKER_ID'] = df_13['JOB_SEEKER_ID'].astype('int64')

print(str(len(df_13)) + ': 13 outcomes len')

df_13['JOB_SEEKER_ID'] = df_13['JOB_SEEKER_ID']


In [ ]:
df_13['JOB_SEEKER_ID'].nunique()
df['JOB_SEEKER_ID'].nunique()

In [ ]:


df = df.rename(columns={'TYPE' : 'DISABILITY_TYPE'})


Large reduction in data size is due to blank values in Num_13 and Den_13

In [ ]:
df_sub = df[['DATE_OF_BIRTH', 'NATIONALITY', 'COUNTRY_OF_BIRTH', 'EDUCATION',
       'EDUCATION_CODE', 'CURRENT_CAPACITY_HOURS', 'JOB_SEEKER_ID', 'GENDER',
       'INTERPRETER_LANGUAGE', 'INTERPRETER_LANGUAGE_CODE',
       'FUNDING_LEVEL_CODE', 'RESIDENTIAL_ADDRESS_LINES',
       'RESIDENTIAL_ADDRESS_SUBURB', 'RESIDENTIAL_ADDRESS_STATE', 'RESIDENTIAL_ADDRESS_POSTCODE',
       'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'RATE_TYPE', 'STATUS', 'CENTRELINK_OUTCOME_CODE',
       'ALLOWANCE_STARTED_ON', 'ALLOWANCE_TYPE', 'ALLOWANCE_RATE',
       'TIME_IN_PLACEMENT', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM',
       'TIME_IN_SITE', 'TIME_IN_PROVIDER', 'MODERATE_INTELLECTUAL_DISABILITY',
       'DISABILITY_TYPE', 'EMPLOYMENT_BENCHMARK']] 



Removing nationality, education, education_code, funding_level_code, centrelink_outcome_code, allowance_started_on, time_in_placement, disability_type_code, employment_benchmark 
- Ref to 'ap_stream_encoded_missing_reasons' excel file for reasons of exclusion

In [ ]:
df_sub = df_sub.drop(columns=['NATIONALITY', 'EDUCATION', 'EDUCATION_CODE', 'FUNDING_LEVEL_CODE', 'CENTRELINK_OUTCOME_CODE', 'ALLOWANCE_STARTED_ON', 'TIME_IN_PLACEMENT', 'EMPLOYMENT_BENCHMARK', 'ALLOWANCE_TYPE', 'ALLOWANCE_RATE'], axis=1)


### Encoding the categorical variables 
- Binary encoding for Y,N variables
- JSCI_PERSONAL_FACTORS_OUTCOME: Nil, Low, Medium, High (0-3)

In [ ]:
# disability_type: 0 for Mental, 1 for Physical
df_sub['DISABILITY_TYPE'] = df_sub['DISABILITY_TYPE'].dropna()
df_sub['DISABILITY_TYPE'] = np.where(df_sub['DISABILITY_TYPE'] == "mental", 0, 1)


In [ ]:
df_sub.columns

In [ ]:
df_sub['COUNTRY_OF_BIRTH'].value_counts()

In [ ]:
col_names = ['DATE_OF_BIRTH', 'COUNTRY_OF_BIRTH',
       'CURRENT_CAPACITY_HOURS', 'JOB_SEEKER_ID', 'GENDER',
       'INTERPRETER_LANGUAGE', 'INTERPRETER_LANGUAGE_CODE',
       'RESIDENTIAL_ADDRESS_LINES', 'RESIDENTIAL_ADDRESS_SUBURB',
       'RESIDENTIAL_ADDRESS_STATE', 'RESIDENTIAL_ADDRESS_POSTCODE',
       'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE',
       'TIME_IN_PROVIDER', 'MODERATE_INTELLECTUAL_DISABILITY']


for x in col_names:
    print('\n ' + str(x) + '\n', df_sub[x].value_counts())

# subsetting the features:
features = ['DATE_OF_BIRTH', 'COUNTRY_OF_BIRTH',
       'CURRENT_CAPACITY_HOURS', 'JOB_SEEKER_ID', 'GENDER',
       'INTERPRETER_LANGUAGE', 'INTERPRETER_LANGUAGE_CODE',
       'RESIDENTIAL_ADDRESS_LINES', 'RESIDENTIAL_ADDRESS_SUBURB',
       'RESIDENTIAL_ADDRESS_STATE', 'RESIDENTIAL_ADDRESS_POSTCODE',
       'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE',
       'TIME_IN_PROVIDER', 'MODERATE_INTELLECTUAL_DISABILITY']




Cleaning the columns

- Finding the Age from DATE_OF_BIRTH - Taking the AGE column 
- Age is from the end period of the Dec 2020 star ratings - at 8/01/21
- Dropping rows with more than 10 NULL column values 

In [ ]:
df_sub = df_sub.dropna(thresh=10, axis=0) 

df_sub['DATE_OF_BIRTH'] = pd.to_datetime(df_sub['DATE_OF_BIRTH']).dt.date

end_date = datetime.strptime('2022/10/07', '%Y/%m/%d')
end_date_year = end_date.year
end_date_month = end_date.month
end_date_day = end_date.day

df_sub['AGE'] = df_sub['DATE_OF_BIRTH'].apply(lambda x: end_date_year - x.year - ((end_date_month, end_date_day) < (x.month, x.day)))
# converting Age -> INT
df_sub['AGE'] = df_sub['AGE'].astype('int64')
df_sub[['DATE_OF_BIRTH', 'AGE']]

In [ ]:

df_sub['INTERPRETER_LANGUAGE'] = df_sub['INTERPRETER_LANGUAGE'].fillna('NOT APPLICABLE')
df_sub['INTERPRETER_LANGUAGE'].value_counts()

In [ ]:
df_sub['MODERATE_INTELLECTUAL_DISABILITY'].value_counts()

In [ ]:
df_sub[['TIME_IN_STREAM',
       'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER']]

df_sub['TIME_IN_STREAM'].isna().sum() # drop missing values (only a few)
df_sub = df_sub[df_sub['TIME_IN_STREAM'].isna() == False]

In [ ]:
df_sub['TIME_IN_PROVIDER'].isna().sum()

In [ ]:
df_sub['TIME_IN_SITE'].isna().sum()

In [ ]:

df_sub['WAGE_SUBSIDY_ELIGIBLE'].value_counts()

In [ ]:
df_sub['WAGE_SUBSIDY_ELIGIBLE'].isna().sum() 

### Encoding 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_SITE'
- WAGE_SUBSIDY_ELIGIBLE: encoded by maintaining the proportion of 1 to 0s 
- TIME_IN_SITE: encoded by the normal distribution of the values 

1. Plotting the distribution of WAGE_SUBSIDY_ELIGIBLE, TIME_IN_SITE

In [ ]:
df_sub.boxplot(column=['TIME_IN_SITE'])

The distribution of TIME_IN_SITE is slightly left skewed, replace missing values with MEAN

In [ ]:
mean_time_site = df_sub['TIME_IN_SITE'].mean()
df_sub['TIME_IN_SITE'] = df_sub['TIME_IN_SITE'].fillna(mean_time_site)


Replacing missing values in WAGE_SUBSIDY_ELIGIBLE with the proportion of existing values
### Part done MANUALLY in EXCEL -> filter on NULL values on WAGE_SUBSIDY_ELIGIBLE + rate_type (wage) + status (approved, etc.)


In [ ]:
df_sub['WAGE_SUBSIDY_ELIGIBLE'].value_counts()

In [ ]:
validRates = ['DES-DMS Wage Start Subsidy Payment', 'DES-DMS Wage Subsidy Fee', 'DES-ESS Wage Start Subsidy Payment', 'DES-ESS Wage Subsidy Fee', 'Restart Wage Subsidy DESA Payment', 'Restart Wage Subsidy DESB Payment', 'Restart Wage Subsidy DESB Payment', 'Restart Wage Subsidy Special Claim-DMS Job Seeker', 'Restart Wage Subsidy Special Claim-ESS Job Seeker']
validStatus = ['Approved', 'Pending', 'Lodged']



In [ ]:
df_sub['WAGE_SUBSIDY_ELIGIBLE'].value_counts()

In [ ]:
df_sub['WAGE_SUBSIDY_ELIGIBLE'].isna().sum() #-> replace with 0 -> assuming the manual replacement in EXCEL has been done

In [ ]:
df_sub['WAGE_SUBSIDY_ELIGIBLE'] = df_sub['WAGE_SUBSIDY_ELIGIBLE'].fillna(0)

In [ ]:
# Cleaning the columns based on distinct values -->

df_sub['COUNTRY_OF_BIRTH'].isna().sum()
# 24 na's -> drop rows
df_sub = df_sub[df_sub['COUNTRY_OF_BIRTH'].notna()]


## INTERPRETER_LANGUAGE: if it is 'NOT APPLICABLE','NaN', 'Sign Language' -> then encode '0', otherwise '1'
no_inter = ['NOT APPLICABLE', 'NaN', 'Sign Language', '*', 'NULL']
df_sub['INTERPRETER_LANGUAGE'] = np.where(df_sub['INTERPRETER_LANGUAGE'].isin(no_inter), 0, 1)



# CULTURAL_linguist from 'COUNTRY_OF_BIRTH', 'INTERPRETER_LANGUAGE'
# if country_of_birth is NOT Australia, then 1 otherwise 0:
df_sub['COUNTRY_DIVERSE'] = np.where(df_sub['COUNTRY_OF_BIRTH'] != 'Australia', 1, 0)
df_sub['CULTURAL_LINGUIST'] = np.where((df_sub['COUNTRY_DIVERSE'] == 1) & (df_sub['INTERPRETER_LANGUAGE'] == 1), 1, 0)




## CURRENT_CAPACITY_HOURS: encode Aug-14 -> "8-14" range - but taking the midpoint  (and check with Depar why it would be Aug-14, should we use imputation?), remove NaN, remove UNKNOWN(count = 6)
# integer encoding with Midpoint (rounded down)

df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('Aug-14', '8-14')
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('NaN', 0)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('0-7', 3)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('8+', 8)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('8-14', 11)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('15-22', 18)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('23-29', 26)
df_sub['CURRENT_CAPACITY_HOURS'] = df_sub['CURRENT_CAPACITY_HOURS'].replace('30+', 30)
df_sub = df_sub[df_sub['CURRENT_CAPACITY_HOURS'] != 'UNNOWN']




## Gender: 0 for Male, 1 for Female
df_sub['GENDER'] = df_sub['GENDER'].replace('Female', 1)
df_sub['GENDER'] = df_sub['GENDER'].replace('Male', 0)


# Type conversions --
## INDIGENOUS_IND: Check that type is numeric 
## WAGE_SUBSIDY_ELIGIBLE: check type is numeric 

dict_types = {'INDIGENOUS_IND' : 'int', 'WAGE_SUBSIDY_ELIGIBLE' : 'int'}

df_sub = df_sub.astype(dict_types)

## MODERATE_INTELLECTUAL_DISABILITY - should include with 'DISABILITY_TYPE_CODE'? Or separately? - encode True, False with 0 and 1
df_sub['MODERATE_INTELLECTUAL_DISABILITY'] = np.where(df_sub['MODERATE_INTELLECTUAL_DISABILITY'] == False, 0, 1) 
# need to drop the 1
df_sub = df_sub[df_sub['MODERATE_INTELLECTUAL_DISABILITY'] != 1]



In [ ]:
df_sub['INTERPRETER_LANGUAGE'].value_counts()

In [ ]:
df_sub['CURRENT_CAPACITY_HOURS'].value_counts()

In [ ]:
df_sub[['RESIDENTIAL_ADDRESS_LINES', 'RESIDENTIAL_ADDRESS_SUBURB', 'RESIDENTIAL_ADDRESS_POSTCODE',
       'RESIDENTIAL_ADDRESS_STATE']]

Identifying the metro postcodes for each state
- Just using postcode and state
- Cleaning the columns

In [ ]:
df_sub['RESIDENTIAL_ADDRESS_STATE'].value_counts()

In [ ]:
# remove '*' in state
df_sub = df_sub[df_sub['RESIDENTIAL_ADDRESS_STATE'] != '*']
df_sub['RESIDENTIAL_ADDRESS_STATE'].value_counts()

# convert postcode to int
df_sub['RESIDENTIAL_ADDRESS_POSTCODE'] = df_sub['RESIDENTIAL_ADDRESS_POSTCODE'].astype('int64')

df_sub = df_sub[df_sub['GENDER'] != 'Unknown'] 

Metro postcodes for each state
source: https://support.shippit.com/hc/en-us/articles/4403703156377-Rule-to-Allocate-a-Courier-for-Metro-and-Regional-Destinations

Source: http://www.impactlists.com.au/ImpactLists/media/list-tools/Useful-Postcode-Ranges.pdf


In [ ]:
### INDIGENOUS_STATUS and METRO LOCATION -> check address postcode AND state
df_sub['METRO'] = False # initialise the metro column to False values


# ind = 0 (start bounds), ind=1 (end bounds)
nsw_met = [[1000, 1920], [2000, 2239], [2555, 2574], [2740, 2786]] # sydney
qld_met = [[2484, 2494], [4000, 4370], [4373, 4381], [4400, 4405], [4500, 4580], [4600, 4610], [4614, 4618], [9000, 9919]] #brisbane
vic_met = [[3000, 3210], [3335, 3341], [3425, 3443], [3750, 3811], [3910, 3920], [3926, 3944], [3972, 3978], [3980, 3983], [8000, 8899]] #Melbourne
wa_met = [[6000, 6214], [6800, 6999]] #perth
tas_met = [[7000,7010], [7249,7250]] # hobart
sa_met = [[5000, 5199], [5800, 5999]] #adelaide
nt_met = [[800, 820], [900, 910]] # darwin
act_met = [[200,299], [2600, 2620], [2900,2920]] #canberra



# creating the "METRO" column - 1 if metro postcode, 0 otherwise
def metro_postcode(postcode_list, postcode):
    is_metro = False # initialise boolean 
    i = 0
    while i < len(postcode_list):
        start_bounds = int(postcode_list[i][0])
        end_bounds = int(postcode_list[i][1])
        if (postcode >= start_bounds) & ( postcode <= end_bounds):
            is_metro = True
            break
        i += 1

    return is_metro




# passing the postcodes to the 'metro_postcode' function ->
for i, row in df_sub.iterrows():
    x = df_sub['RESIDENTIAL_ADDRESS_POSTCODE'][i] #postcode
    if x is None:
        pass 
    else: 
        # getting the state
        state = df_sub['RESIDENTIAL_ADDRESS_STATE'][i]
        if state == 'NSW':
            df_sub['METRO'][i] = metro_postcode(nsw_met, x)
        elif state == 'QLD':
            df_sub['METRO'][i] = metro_postcode(qld_met, x)
        elif state == 'VIC':
            df_sub['METRO'][i] = metro_postcode(vic_met, x)
        elif state == 'WA':
            df_sub['METRO'][i] = metro_postcode(wa_met, x)
        elif state == 'TAS':
            df_sub['METRO'][i] = metro_postcode(tas_met, x)
        elif state == 'SA':
            df_sub['METRO'][i] = metro_postcode(sa_met, x)
        elif state == 'NT':
            df_sub['METRO'][i] = metro_postcode(nt_met, x)
        elif state == 'ACT':
            df_sub['METRO'][i] = metro_postcode(act_met, x)
        else: 
            pass



In [ ]:
df_sub[["RESIDENTIAL_ADDRESS_POSTCODE", "RESIDENTIAL_ADDRESS_STATE", "METRO"]].head(5)
# encode True -> 1, False -> 0
df_sub["METRO"] = np.where(df_sub["METRO"] == True, 1, 0)

# filtering for INDIGENOUS_INDICATOR = 1 and METRO = 1
df_sub['IND_METRO'] = np.where((df_sub['INDIGENOUS_IND'] == 1) & (df_sub['METRO'] == 1), 1, 0) 

df_sub[["RESIDENTIAL_ADDRESS_POSTCODE", "RESIDENTIAL_ADDRESS_STATE", "METRO", "INDIGENOUS_IND", "IND_METRO"]].head(10)

In [ ]:
# Checking missing values again: 
naVal = df_sub.isna().sum() * 100 / len(df)
missValDf = pd.DataFrame({'column_name' : df_sub.columns, 'percent_missing' : naVal})
missValDf.sort_values(by=['percent_missing'], ascending=False)

In [ ]:
# dropping the current_capacity_hours missing values
df_sub = df_sub[df_sub['CURRENT_CAPACITY_HOURS'].isna() == False]

In [ ]:
naVal = df_sub.isna().sum() * 100 / len(df)
missValDf = pd.DataFrame({'column_name' : df_sub.columns, 'percent_missing' : naVal})
missValDf.sort_values(by=['percent_missing'], ascending=False)

### Final variables subset for logistic regression model

In [ ]:
features = ['CURRENT_CAPACITY_HOURS', 'GENDER', 'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER',
       'MODERATE_INTELLECTUAL_DISABILITY', 'AGE', 'COUNTRY_DIVERSE', 'CULTURAL_LINGUIST', 'IND_METRO']


# Identifying strong predictors for numerating participants

Using StandardScalar to standardise the data

In [ ]:
#PCA
features = ['CURRENT_CAPACITY_HOURS', 'GENDER', 'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER',
       'MODERATE_INTELLECTUAL_DISABILITY', 'AGE', 'COUNTRY_DIVERSE', 'CULTURAL_LINGUIST', 'IND_METRO']


### Correlation Matrix
Between numeric variables

In [ ]:
df_sub.dtypes

Checking whether there exists variable with all 0s or 1s -> to avoid PerfectSeparation error in the logistic regression

In [ ]:
#for col in df_sub:
#    print(col , df_sub[col].unique())

In [ ]:
df_feat = df_sub[['CURRENT_CAPACITY_HOURS', 'GENDER', 'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER',
       'MODERATE_INTELLECTUAL_DISABILITY', 'AGE', 'COUNTRY_DIVERSE', 'CULTURAL_LINGUIST', 'IND_METRO']]
df_feat.corr()

Dropping the NaN correlation variables

In [ ]:
df_feat =  df_sub[['CURRENT_CAPACITY_HOURS', 'GENDER', 'DISABILITY_TYPE', 'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER',
       'AGE', 'COUNTRY_DIVERSE', 'CULTURAL_LINGUIST', 'IND_METRO']]

corr_matrix = df_feat.corr()
corr_matrix.style.background_gradient(cmap='Blues')

### Relationships between Features Chart

In [ ]:
import seaborn as sns
df_feat_num =  df_sub[['CURRENT_CAPACITY_HOURS', 'GENDER', 'DISABILITY_TYPE', 'INDIGENOUS_IND', 'WAGE_SUBSIDY_ELIGIBLE', 'TIME_IN_STREAM', 'TIME_IN_PROGRAM', 'TIME_IN_SITE', 'TIME_IN_PROVIDER',
       'AGE', 'COUNTRY_DIVERSE', 'CULTURAL_LINGUIST', 'IND_METRO']]
sns.pairplot(data = df_feat_num, diag_kind='kde')
plt.show()

### Reasons for Low Accuracy in Model (13 week outcomes)
from analysis of sns pairplot:

- Across the diagonal where each feature is plotted against itself, the distribution of each feature differs.  
- IND_METRO: no strong relations to Num_13. When indicator is 0 and 1, Num_13 = 0, and left skewed 
- CULTURAL_LINGUIST: no strong relation, and left skewed
- COUNTRY_DIVERSE: no strong relation 
- None of the variables are strong indicators for Num_13. 
However, we can reduce correlation between the variables and reduce dimensionality. 
Note: Since Num_13 is imbalanced with Num_13 = 1 as the majority class -> then there may be a slight right-hand skew against the features 

- TIME_IN_PROGRAM and TIME_IN_STREAM -> positive correlation (REMOVE ONE)
- TIME_IN_SITE and TIME_IN_PROVIDER -> positive correlation (REMOVE ONE)
- AGE is right skewed
- Two peaks (0, 1) with binary features 
- WAGE_SUBSIDY_ELIGIBLE -> left skewed
- INDIGEOUS_IND -> left skewed

Subsetting the features and reducing dependency between these. Final subset: 

features = ['CURRENT_CAPACITY_HOURS', 'GENDER', 'INDIGENOUS_IND', 'DISABILITY_TYPE', 'TIME_IN_PROGRAM', 'TIME_IN_SITE',
       'AGE']

### Binary Logistic Regression Model (Num_13 = 0 or 1)
Dependent var: 0 or 1 (Num_13)
Independent: features

In [ ]:
features = ['CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'AGE', 'IND_METRO']


Across Diagonal -> correct predictions for Binary output (0, 1)


In [ ]:
from sklearn.metrics import classification_report


- f1-score (comb. of precision and recall is 0.81 for both classes)
- support = no. of actual occurrences of the class in the specified dataset

Coefficients for the Model

### Final LG equation - for a person to record an outcome (Num_13 = 1):

logit(p) = -0.154(AGE_AT_EXTRACT) - 1.394(WEEKS_REGISTERED) - 1.184(TEMP_CAPACITY_2_HOURS) + 0.375(CURRENT_CAPACITY_HOURS) - 0.446(FUTURE_CAPACITY_INTERVENTION_HOURS) -0.359(JSCI_PERSONAL_FACTORS_OUTCOME) - 1.536(CALD) - 0.348



### Stat Regression Function (13, 26, 52 week)

### Improve performance - GridSearch for hyperparameter tuning for Logistic Regression model 

### Passing each job seeker row -> eval regression function

In [ ]:
df_13['Num_13'].value_counts()


In [ ]:
x = 233/1030
x # defines the ratio of class weights
1 - x

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report


def eval_regression(df_n, df_apj, features, n_type):
    '''Evaluation of the statistical regression fuction. Passes each performance measure dataframe (from pre-quarterisation) and predicts whether a participant numerates based on
    demographic variables. Returns the classification metric report, confusion matrix and the logreg equation. Input: takes df_n (df_13, df_26, or df_52), and df_apj (stream_participant and jobseeker), 'n' : string of the performance measure,
    df_upsampled: dataframe for the upsampled minority class of df_n (if class imbalance - otherwise None).
    df_apj is the equivalent to the df_sub that has been pre-processed prior and outside of the function'''
    df_n = df_n.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
    df_n['JOB_SEEKER_ID'] = df_n['JOB_SEEKER_ID'].astype('int64')
    # merging df_n and df_apj:
    
    df_merge = df_apj.merge(df_n, on=['JOB_SEEKER_ID'], how='inner')
    df_merge['JOB_SEEKER_ID'] = df_merge['JOB_SEEKER_ID'].astype('int64')
    # check class imbalance:
    df_sub = df_merge
    num_n = str(n_type)
   
    
    

    # running the Binary Logistic Regression (Num_n = 0, or Num_n = 1)
    #separation of the features:
    x = df_sub.loc[:, features].values
    # target (Num_13)
    y = df_sub[num_n]
    
 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16, stratify=y)
    # X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)
    
    smote = SMOTE(sampling_strategy=1, random_state=12)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    scaler = StandardScaler()
    # scaling the features
    X_train = scaler.fit_transform(X_resampled)
    X_test = scaler.transform(X_test)
    

    # HYPERPARAMETER TUNING with GridSearchCV
    params = {'C': [0.01, 0.1, 1, 10, 100]}

    clf = LogisticRegression(penalty='l2', class_weight='balanced', random_state=42)


    folds = 5
    model_cv = GridSearchCV(estimator = clf, 
                            param_grid = params, 
                            scoring= 'accuracy', 
                            cv = folds,
                            return_train_score=True,
                            verbose = 3)

    model_cv.fit(X_resampled, y_resampled) # fit on training
    # getting the best hyper-parameter
    C = model_cv.best_params_
    hyperVal = C.get('C')



    # define model 
    logreg = LogisticRegression(C=hyperVal, penalty='l2', class_weight='balanced', random_state=42)



    # fit the model and predict
    model = logreg.fit(X_resampled, y_resampled)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Adjust the decision threshold
    threshold = 0.5  # You can adjust this value based on your needs
    y_pred = (y_pred_proba >= threshold).astype(int)

    # confusion matrix:
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

    #printing test set results
    class_report = classification_report(y_test, y_pred)

    # print the AUC:
    y_pred_proba = logreg.predict_proba(X_train)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_resampled,  y_pred_proba)
    auc = metrics.roc_auc_score(y_resampled, y_pred_proba)
    ### AUC - COMMENTED OUT
    #plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
    #plt.legend(loc=4)
    #plt.show()
    

    # the logistic regression equation 
    
    coef = model.coef_
    coef = list(coef)
    intercept = model.intercept_
    df_merge = df_merge[['JOB_SEEKER_ID','CURRENT_CAPACITY_HOURS', 'GENDER', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE', 'INDIGENOUS_IND', 'IND_METRO', 'WAGE_SUBSIDY_ELIGIBLE']]
    type_dict = {'CURRENT_CAPACITY_HOURS' : float, 'GENDER' : float, 'TIME_IN_SITE' : float, 'DISABILITY_TYPE' : float, 'AGE' : float, 'INDIGENOUS_IND' : float, 'IND_METRO' : float}
    df_merge = df_merge.astype(type_dict)
    return cnf_matrix, class_report, coef, intercept, df_merge, model

### 13, 26, 52 week stat regression outcomes

1. Model coefficients would differ based on each performance measure
2. Generate the model coefficients, get from the eval_regression function. Create a formula.
3. Pass each performance measure dataframe (df_n) 

### 13 week outcomes to regression function
- Using model_13 -> pass each row of the dataframe to the model and call "model.pred(row)" -> set this to expected 

In [ ]:
df_sub

In [ ]:
features =['CURRENT_CAPACITY_HOURS', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']

In [ ]:
df_13

In [ ]:
cnf_matrix_13, class_report_13, coef_13, intercept_13, df_merge_13, model_13 = eval_regression(df_13, df_sub, features, 'Num_13')
print(class_report_13)


In [ ]:
print(class_report_13)

### KNN 

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
%matplotlib inline
def knn_regression(df_n, df_apj, features, n_type):
    '''Evaluation of the statistical regression fuction. Passes each performance measure dataframe (from pre-quarterisation) and predicts whether a participant numerates based on
    demographic variables. Returns the classification metric report, confusion matrix and the logreg equation. Input: takes df_n (df_13, df_26, or df_52), and df_apj (stream_participant and jobseeker), 'n' : string of the performance measure,
    df_upsampled: dataframe for the upsampled minority class of df_n (if class imbalance - otherwise None).
    df_apj is the equivalent to the df_sub that has been pre-processed prior and outside of the function'''
    df_n = df_n.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
    df_n['JOB_SEEKER_ID'] = df_n['JOB_SEEKER_ID'].astype('int64')
    # merging df_n and df_apj:
    
    df_merge = df_apj.merge(df_n, on=['JOB_SEEKER_ID'], how='inner')
    df_merge['JOB_SEEKER_ID'] = df_merge['JOB_SEEKER_ID'].astype('int64')
    # check class imbalance:
    df_sub = df_merge
    num_n = str(n_type)
   
    
    # running the Binary Logistic Regression (Num_n = 0, or Num_n = 1)
    #separation of the features:
    x = df_sub.loc[:, features].values
    # target (Num_13)
    y = df_sub[num_n]
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)
    rus = RandomOverSampler(sampling_strategy='minority', random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train, y_train) # only resample after train/test split
    # creating a balanced training dataset -> # scale features
    scaler = StandardScaler()
    
    X_train = scaler.fit_transform(X_resampled)
    X_test = scaler.transform(X_test)
    X_resampled = X_train  
    

    
    # define model 
    knn_model = KNeighborsClassifier()

    # create the hyperparameter grid
    param_grid = {
        'n_neighbors': range(1, 21),
        'metric': ['minkowski'],
        'p': [1, 2]
    }
    # create the cross-validation method
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

    # perform the grid search
    grid_search = GridSearchCV(knn_model, param_grid, scoring='accuracy', cv=cv, n_jobs=-1)
    grid_search.fit(X_resampled, y_resampled)

    # get the best model
    best_model = grid_search.best_estimator_
    
     # predict on the test set
    y_pred = best_model.predict(X_test)

    # print the classification report
    print(accuracy_score(y_test, y_pred))

    # print the confusion matrix
    print(confusion_matrix(y_test, y_pred))
    
    return best_model, X_train, X_test, y_resampled, y_test, scaler

In [ ]:
features = ['WAGE_SUBSIDY_ELIGIBLE', 'CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'AGE']

In [ ]:
model_13, X_train, X_test, y_resampled, y_test, scaler = knn_regression(df_13, df_sub, features, 'Num_13')

#### Finding the appropriate values for k 
Through exploration of the elbow graph, the ideal number of neighbours with the lowest RMSE is ~8. Neighbours at 15 generates a lower accuracy.

### Generating expected outcomes for 13 week 
- Addition of a 'Num_13_exp' -> Numerator for 13 week EXPECTED column in the "df_13" one
- Using model_13 -> pass each row of the dataframe to the model and call "model.pred(row)" -> set this to expected 
- Passing all rows (1263) of df_13 t

In [ ]:
model_13

In [ ]:
# set df_13['JOB_SEEKER_ID'] as the index
df_13 = df_13.set_index('JOB_SEEKER_ID')
# create Num_13_exp column
df_13['Num_13_exp'] = 0

In [ ]:
df_13 = df_13.drop(columns={'Unnamed: 0'})

In [ ]:
df_13.value_counts()

In [ ]:
df_merge_13

In [ ]:
# merge df_13 with df_merge_13
df_13_final = df_merge_13.merge(df_13, on=['JOB_SEEKER_ID'], how='inner')
df_13_final = df_13_final.rename(columns={'Num_13' : 'Num_13_full', 'Den_13' : 'Den_13_full'})

In [ ]:
df_13_final = df_13_final[["JOB_SEEKER_ID", "Num_13_full", "Den_13_full"]]
df_13 = df_13_final
df_13['Num_13_exp'] = 0 #initilalise expected outcomes

In [ ]:
print(features)


Given the class imbalance -> resample, when predicting, to offset the prediction of all 0s

In [ ]:
df_merge_13

In [ ]:
df_13

#### Output file for 13 week expected outcomes

In [ ]:
df_13.to_csv('df_13_exp.csv')

df_13['Num_13_exp'].value_counts()

Merging df_n with df_merge_n - to get common rows

### Passing the 26 week outcome to the stat_regression function 
- Creating 3 subset dataframes for 3 performance subtypes: Full, Pathway, Work Assist/Bonus\
- Removing NaN values

In [ ]:
# 26 week outcome
df_26 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_26_new.csv")
df_26_full = df_26[["Job_Seeker_ID", "Num_26_full", "Den_26_full"]]
df_26_full = df_26_full.dropna()
df_26_path = df_26[["Job_Seeker_ID", "Num_26_path", "Den_26_path"]]
df_26_path = df_26_path.dropna()
df_26_wrkast = df_26[["Job_Seeker_ID", "Num_26_wrkast", "Den_26_wrkast"]]
df_26_wrkast = df_26_wrkast.dropna()

In [ ]:
df_26


In [ ]:
df_26_full['Num_26_full'].value_counts()

In [ ]:
df_26_path['Num_26_path'].value_counts()

In [ ]:
df_26_wrkast['Num_26_wrkast'].value_counts()

In [ ]:
cnf_matrix_26, class_report_26, coef_26, intercept_26, df_merge_26, model_26 = eval_regression(df_26_full, df_sub, features, 'Num_26_full')


print("26 week FULL outcome" + '\n')
print(cnf_matrix_26)
print(class_report_26)


### Since there is only one class for 26 week work assist/pathway - use the model for the full outcome for prediction



##### 26 week full outcome (expected)

In [ ]:
df_26_full = df_26_full.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
# merge with df_merge_26
df_26_full_m = df_merge_26.merge(df_26_full, on=['JOB_SEEKER_ID'], how='inner')

df_merge_26 = df_26_full_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO"]]

df_merge_26 = df_merge_26.set_index('JOB_SEEKER_ID')

In [ ]:
df_26_full_m
df_26_full = df_26_full_m[["JOB_SEEKER_ID", "Num_26_full", "Den_26_full"]]
df_26_full['Num_26_full_exp'] = 0
df_26_full = df_26_full.set_index('JOB_SEEKER_ID')

In [ ]:
for index, row in df_merge_26.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'GENDER', 'TIME_IN_SITE', 'DISABILITY_TYPE']])
    X_test = X_test.values.reshape(1, -1)
    if index == 0:
        pass
    else:
        df_26_full.loc[index].Num_26_full_exp = model_26.predict(X_test)
    

### Passing 52 performance measure to regression function
- Since only 1 class in 52 week outcome -> using the 26 week prediction function 
- Replace line 9 when more data comes in.

##### 52 week full outcome (expected)

In [ ]:
df_52 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_52_new.csv")
df_52_full = df_52[["Job_Seeker_ID", "Num_52_full", "Den_52_full"]]

df_52_path = df_52[["Job_Seeker_ID", "Num_52_path", "Den_52_path"]]


df_52_full = df_52_full.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
# merge with df_merge_52
df_52_full_m = df_sub.merge(df_52_full, on=['JOB_SEEKER_ID'], how='inner')

df_merge_52 = df_52_full_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO"]]

df_merge_52 = df_merge_52.set_index('JOB_SEEKER_ID')

### Random Forest for regression - To-Do

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import cross_val_score


def rf_model(df_n, df_apj, features, n_type):
    '''Evaluation of the statistical regression fuction. Passes each performance measure dataframe (from pre-quarterisation) and predicts whether a participant numerates based on
    demographic variables. Returns the classification metric report, confusion matrix and the logreg equation. Input: takes df_n (df_13, df_26, or df_52), and df_apj (stream_participant and jobseeker), 'n' : string of the performance measure,
    df_upsampled: dataframe for the upsampled minority class of df_n (if class imbalance - otherwise None).
    df_apj is the equivalent to the df_sub that has been pre-processed prior and outside of the function'''
    df_n = df_n.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
    df_n['JOB_SEEKER_ID'] = df_n['JOB_SEEKER_ID'].astype('int64')
    # merging df_n and df_apj:
    
    df_merge = df_apj.merge(df_n, on=['JOB_SEEKER_ID'], how='inner')
    df_merge['JOB_SEEKER_ID'] = df_merge['JOB_SEEKER_ID'].astype('int64')
    # check class imbalance:
    df_sub = df_merge
    num_n = str(n_type)
   
    
    

    # running the Binary Logistic Regression (Num_n = 0, or Num_n = 1)
    #separation of the features:
    x = df_sub.loc[:, features].values
    # target (Num_13)
    y = df_sub[num_n]
    
 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16, stratify=y)
    # X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)
    
    adasyn = ADASYN(sampling_strategy=1, random_state=12)
    X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

    scaler = StandardScaler()
  
    # Standardize the training and testing datasets
    X_train = scaler.fit_transform(X_resampled)
    X_test = scaler.transform(X_test)


        # HYPERPARAMETER TUNING with RandomizedSearchCV
    params = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [10, 20, 30, 40, 50, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    clf = RandomForestClassifier(random_state=42)

    # Using RandomizedSearchCV for a faster search
    model_cv = RandomizedSearchCV(estimator=clf, 
                                  param_distributions=params, 
                                  n_iter=50,
                                  scoring='accuracy', 
                                  cv=5,
                                  verbose=3, 
                                  random_state=42, 
                                  n_jobs=-1)

    model_cv.fit(X_resampled, y_resampled)
    best_params = model_cv.best_params_

    # define model
    rf_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                      max_features=best_params['max_features'],
                                      max_depth=best_params['max_depth'],
                                      min_samples_split=best_params['min_samples_split'],
                                      min_samples_leaf=best_params['min_samples_leaf'],
                                      bootstrap=best_params['bootstrap'],
                                      random_state=42)

    # fit the model and predict
    rf_model.fit(X_resampled, y_resampled)
    y_pred = rf_model.predict(X_test)

    # Perform cross-validation to evaluate the model
    cv_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5, scoring='accuracy')
    print('Cross-validation scores:', cv_scores)
    print('Mean cross-validation score:', np.mean(cv_scores))
    
        # confusion matrix:
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

    #printing test set results
    class_report = classification_report(y_test, y_pred)

    

    # the logistic regression equation 
    
  

    df_merge = df_merge[['JOB_SEEKER_ID','CURRENT_CAPACITY_HOURS', 'GENDER', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE', 'INDIGENOUS_IND', 'IND_METRO']]
    type_dict = {'CURRENT_CAPACITY_HOURS' : float, 'GENDER' : float, 'TIME_IN_SITE' : float, 'DISABILITY_TYPE' : float, 'AGE' : float, 'INDIGENOUS_IND' : float, 'IND_METRO' : float}
    df_merge = df_merge.astype(type_dict)
    
    return cnf_matrix, class_report, df_merge, rf_model
    


### Random Forest - 13, 26, 52 weeks

In [ ]:
features_sub = ['CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE']

In [ ]:
cnf_matrix_13, class_report_13, df_merge_13, rf_model_13 = rf_model(df_13, df_sub, features_sub, 'Num_13_full')

In [ ]:
print(cnf_matrix_13)

In [ ]:
# passing each dataframe row of df_merge_13 to the model.pred()
for index, row in df_merge_13.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    X_test = scaler.fit_transform(X_test)
    df_13.loc[index, 'Num_13_exp'] = rf_model_13.predict(X_test)
    

In [ ]:
df_13.Num_13_exp.value_counts()

### Ensemble Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline
def en_model(df_n, df_apj, features, n_type):
    '''Evaluation of the statistical regression fuction. Passes each performance measure dataframe (from pre-quarterisation) and predicts whether a participant numerates based on
    demographic variables. Returns the classification metric report, confusion matrix and the logreg equation. Input: takes df_n (df_13, df_26, or df_52), and df_apj (stream_participant and jobseeker), 'n' : string of the performance measure,
    df_upsampled: dataframe for the upsampled minority class of df_n (if class imbalance - otherwise None).
    df_apj is the equivalent to the df_sub that has been pre-processed prior and outside of the function'''
    df_n = df_n.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
    df_n['JOB_SEEKER_ID'] = df_n['JOB_SEEKER_ID'].astype('int64')
    # merging df_n and df_apj: Participants feature data with their num/den counts
    
    df_merge = df_apj.merge(df_n, on=['JOB_SEEKER_ID'], how='inner')
    df_merge['JOB_SEEKER_ID'] = df_merge['JOB_SEEKER_ID'].astype('int64')
    # check class imbalance:
    df_sub = df_merge
    num_n = str(n_type)
   
    
    

    # running the Binary Logistic Regression (Num_n = 0, or Num_n = 1)
    #separation of the features:
    x = df_sub.loc[:, features].values
    # target (Num_13)
    y = df_sub[num_n]
    
 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16, stratify=y)
    # X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)
    
    adasyn = ADASYN(sampling_strategy=1, random_state=12)

    rus = RandomUnderSampler(sampling_strategy=1, random_state=12)

    # Create pipeline to apply both over-sampling and under-sampling
    pipeline = Pipeline([('over', adasyn), ('under', rus)])
    X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)


  
    # Standardize the training and testing datasets
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_resampled)
    X_test = scaler.transform(X_test)

     # Hyperparameter tuning for logistic regression
    params = {'C':[0.01, 0.1, 1, 10, 100]}
    clf = LogisticRegression(penalty='l2', random_state=42)
    folds = 5
    model_cv = GridSearchCV(estimator=clf, 
                            param_grid=params, 
                            scoring='accuracy', 
                            cv=folds,
                            return_train_score=True,
                            verbose=3)
    model_cv.fit(X_resampled, y_resampled)
    hyperVal = model_cv.best_params_['C']

   
    # Hyperparameter tuning for random forest
    params = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [10, 20, 30, 40, 50, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    clf = RandomForestClassifier(random_state=42)

    model_rf_cv = RandomizedSearchCV(estimator=clf, 
                                  param_distributions=params, 
                                  n_iter=50,
                                  scoring='accuracy', 
                                  cv=5,
                                  verbose=3, 
                                  random_state=42, 
                                  n_jobs=-1)

    model_rf_cv.fit(X_resampled, y_resampled)
    best_params = model_rf_cv.best_params_

    # Instantiate the individual models
    logreg = LogisticRegression(C=hyperVal, penalty='l2', random_state=42)
    rf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                max_features=best_params['max_features'],
                                max_depth=best_params['max_depth'],
                                min_samples_split=best_params['min_samples_split'],
                                min_samples_leaf=best_params['min_samples_leaf'],
                                bootstrap=best_params['bootstrap'],
                                random_state=42)
    svm = SVC(probability=True, random_state=42)

    # Create the ensemble model using VotingClassifier
    ensemble = VotingClassifier(estimators=[('lr', logreg), ('rf', rf), ('svm', svm)], voting='soft')

    # Fit the ensemble model
    ensemble.fit(X_resampled, y_resampled)
    
     # Make predictions
    y_pred = ensemble.predict(X_test)

    # Evaluate the model
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)


    
    # Perform cross-validation to evaluate the model
    cv_scores = cross_val_score(ensemble, X_resampled, y_resampled, cv=5, scoring='accuracy')
    print('Cross-validation scores:', cv_scores)
    print('Mean cross-validation score:', np.mean(cv_scores))
    
    
  

    df_merge = df_merge[['JOB_SEEKER_ID','CURRENT_CAPACITY_HOURS', 'GENDER', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE', 'INDIGENOUS_IND', 'IND_METRO']]
    type_dict = {'CURRENT_CAPACITY_HOURS' : float, 'GENDER' : float, 'TIME_IN_SITE' : float, 'DISABILITY_TYPE' : float, 'AGE' : float, 'INDIGENOUS_IND' : float, 'IND_METRO' : float}
    df_merge = df_merge.astype(type_dict)
    
    return cnf_matrix, class_report, df_merge, ensemble
    


In [ ]:
features_sub = ['CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE']

In [89]:
cnf_matrix_13, class_report_13, df_merge_13, en_model_13 = en_model(df_13, df_sub, features_sub, 'Num_13_full')
# passing each dataframe row of df_merge_13 to the model.pred()


Cross-validation scores: [0.6609589  0.66438356 0.6130137  0.64948454 0.60824742]
Mean cross-validation score: 0.6392176246292897


In [90]:
print(cnf_matrix_13)
print(class_report_13)

[[ 98  99]
 [157 156]]
              precision    recall  f1-score   support

           0       0.38      0.50      0.43       197
           1       0.61      0.50      0.55       313

    accuracy                           0.50       510
   macro avg       0.50      0.50      0.49       510
weighted avg       0.52      0.50      0.50       510



XGBoost Model - Another Ensemble Method

In [91]:
for index, row in df_merge_13.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    X_test = scaler.fit_transform(X_test)
    df_13.loc[index, 'Num_13_exp'] = en_model_13.predict(X_test)

df_13.Num_13_exp.value_counts()


0    1698
Name: Num_13_exp, dtype: int64

### XGBoost Model

In [92]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

def xgboost_model(df_n, df_apj, features, n_type):
    df_n = df_n.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
    df_n['JOB_SEEKER_ID'] = df_n['JOB_SEEKER_ID'].astype('int64')
    # merging df_n and df_apj: Participants feature data with their num/den counts
    
    df_merge = df_apj.merge(df_n, on=['JOB_SEEKER_ID'], how='inner')
    df_merge['JOB_SEEKER_ID'] = df_merge['JOB_SEEKER_ID'].astype('int64')
    # check class imbalance:
    df_sub = df_merge
    num_n = str(n_type)
   
    
    

    # running the Binary Logistic Regression (Num_n = 0, or Num_n = 1)
    #separation of the features:
    x = df_sub.loc[:, features].values
    # target (Num_13)
    y = df_sub[num_n]
    
 
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16, stratify=y)
    # X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=16)
    
    adasyn = ADASYN(sampling_strategy=1, random_state=12)

    rus = RandomUnderSampler(sampling_strategy=1, random_state=12)

    # Create pipeline to apply both over-sampling and under-sampling
    pipeline = Pipeline([('over', adasyn), ('under', rus)])
    X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)

    # Standardize the training and testing datasets
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_resampled)
    X_test = scaler.transform(X_test)


    # Hyperparameter tuning for XGBoost
    params = {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 6, 9, 12],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0, 0.1, 0.2],  # Added regularization parameter
        'reg_lambda': [0.8, 1, 1.2]  # Added regularization parameter
    }

    clf = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

    model_cv = RandomizedSearchCV(estimator=clf,
                                  param_distributions=params,
                                  n_iter=50,
                                  scoring='accuracy',
                                  cv=5,
                                  verbose=3,
                                  random_state=42,
                                  n_jobs=-1)

    model_cv.fit(X_resampled, y_resampled)
    best_params = model_cv.best_params_

    # Instantiate XGBoost with best parameters
    xgb_clf = xgb.XGBClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                min_child_weight=best_params['min_child_weight'],
                                subsample=best_params['subsample'],
                                gamma=best_params['gamma'],
                                colsample_bytree=best_params['colsample_bytree'],
                                reg_alpha=best_params['reg_alpha'],  # Added regularization parameter
                                reg_lambda=best_params['reg_lambda'],  # Added regularization parameter
                                random_state=42, use_label_encoder=False, eval_metric='logloss')

    # Fit the model with early stopping
    xgb_clf.fit(X_resampled, y_resampled, early_stopping_rounds=10, eval_set=[(X_test, y_test)])


    # Make predictions
    y_pred = xgb_clf.predict(X_test)

     # Evaluate the model
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)


    
   
    
    
  

    df_merge = df_merge[['JOB_SEEKER_ID','CURRENT_CAPACITY_HOURS', 'GENDER', 'TIME_IN_SITE', 'DISABILITY_TYPE', 'AGE', 'INDIGENOUS_IND', 'IND_METRO']]
    type_dict = {'CURRENT_CAPACITY_HOURS' : float, 'GENDER' : float, 'TIME_IN_SITE' : float, 'DISABILITY_TYPE' : float, 'AGE' : float, 'INDIGENOUS_IND' : float, 'IND_METRO' : float}
    df_merge = df_merge.astype(type_dict)
    
    return cnf_matrix, class_report, df_merge, xgb_clf



In [124]:
features_sub = ['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']

In [125]:
cnf_matrix_13, class_report_13, df_merge_13, xg_model_13 = xgboost_model(df_13, df_sub, features_sub, 'Num_13_full')
# passing each dataframe row of df_merge_13 to the model.pred()

KeyError: 'Num_13_full'

In [95]:
print(class_report_13)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       197
           1       0.61      1.00      0.76       313

    accuracy                           0.61       510
   macro avg       0.31      0.50      0.38       510
weighted avg       0.38      0.61      0.47       510



#### KNN 13 week outcomes

In [126]:
df_13 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_13_pre.csv")
df_13_full = df_13[["Job_Seeker_ID", "Num_13", "Den_13"]]
df_13_full = df_13_full.dropna()

df_13_full = df_13_full.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_13_full['Num_13_full_exp'] = 0
# merge with df_merge_13
df_13_full_m = df_sub.merge(df_13_full, on=['JOB_SEEKER_ID'], how='inner')

df_13_full_m = df_13_full_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO", "WAGE_SUBSIDY_ELIGIBLE"]]
df_13_full_m = df_13_full_m.set_index(keys=['JOB_SEEKER_ID'])


In [127]:
best_model, X_train, X_test, y_resampled, y_test, scaler_knn_13_full = knn_regression(df_13_full, df_sub, features_sub, 'Num_13')

0.5069033530571992
[[135  79]
 [171 122]]


In [128]:
df_13_full_m

,CURRENT_CAPACITY_HOURS,GENDER,TIME_IN_SITE,DISABILITY_TYPE,AGE,INDIGENOUS_IND,IND_METRO,WAGE_SUBSIDY_ELIGIBLE
JOB_SEEKER_ID,,,,,,,,
1.684700e+05,11,1,800.0,1,22,0,0,1
3.091045e+09,8,0,530.0,1,26,0,0,0
9.690339e+09,11,1,1009.0,1,51,0,0,0
2.641014e+09,18,1,106.0,1,37,0,0,0
3.549240e+06,18,1,1223.0,0,37,0,0,1
...,...,...,...,...,...,...,...,...
2.878234e+09,18,0,581.0,1,54,0,0,1
8.332235e+09,18,1,530.0,1,44,0,0,1
8.562966e+09,3,0,552.0,1,55,0,0,0


In [129]:
df_13_full = df_13_full.set_index(keys=['JOB_SEEKER_ID'])

for index, row in df_13_full_m.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    
    # Apply the same scaler used during training
    X_test_scaled = scaler_knn_13_full.transform(X_test)
    
    df_13_full.loc[index, 'Num_13_full_exp'] = best_model.predict(X_test_scaled)

df_13_full

,Num_13,Den_13,Num_13_full_exp
JOB_SEEKER_ID,,,
9386301003,0,1.0,0
8653691003,0,1.0,1
7870290004,0,1.0,0
6498977509,1,1.0,1
6638331004,0,1.0,0
...,...,...,...
2836891309,1,1.0,0
8771045004,0,1.0,0
15325790,1,1.0,1


### KNN 26 week outcomes

In [130]:
# 26 week outcome
### 26 week
df_26 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_26_new.csv")
df_26_full = df_26[["Job_Seeker_ID", "Num_26_full", "Den_26_full"]]
df_26_full = df_26_full.dropna()

df_26_full = df_26_full.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_26_full['Num_26_full_exp'] = 0
# merge with df_merge_26
df_26_full_m = df_sub.merge(df_26_full, on=['JOB_SEEKER_ID'], how='inner')

df_26_full_m = df_26_full_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO", "WAGE_SUBSIDY_ELIGIBLE"]]



df_26_path = df_26[["Job_Seeker_ID", "Num_26_path", "Den_26_path"]]
df_26_path = df_26_path.dropna()
df_26_path = df_26_path.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_26_path['Num_26_path_exp'] = 0
# merge with df_merge_26
df_26_path_m = df_sub.merge(df_26_path, on=['JOB_SEEKER_ID'], how='inner')

df_26_path_m = df_26_path_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO", "WAGE_SUBSIDY_ELIGIBLE"]]


df_26_wrkast = df_26[["Job_Seeker_ID", "Num_26_wrkast", "Den_26_wrkast"]]
df_26_wrkast = df_26_wrkast.dropna()
df_26_wrkast['Num_26_wrkast_exp'] = 0

In [137]:
features_sub = ['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']

In [138]:
best_model, X_train, X_test, y_resampled, y_test, scaler_knn_26_full = knn_regression(df_26_full, df_sub, features_sub, 'Num_26_full')

0.615686274509804
[[245  86]
 [110  69]]


In [139]:
df_26_full_m = df_26_full_m.set_index(keys=['JOB_SEEKER_ID'])
df_26_full = df_26_full.set_index(keys=['JOB_SEEKER_ID'])
for index, row in df_26_full_m.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    
    # Apply the same scaler used during training
    X_test_scaled = scaler_knn_26_full.transform(X_test)
    
    df_26_full.loc[index, 'Num_26_full_exp'] = best_model.predict(X_test_scaled)

df_26_full


,Num_26_full,Den_26_full,Num_26_full_exp
JOB_SEEKER_ID,,,
9386301003,0.0,0.0,0
8653691003,0.0,1.0,0
7870290004,0.0,1.0,0
6498977509,1.0,1.0,1
6638331004,1.0,1.0,0
...,...,...,...
2836891309,0.0,0.0,1
8771045004,0.0,1.0,0
15325790,0.0,0.0,0


In [140]:
best_model, X_train, X_test, y_resampled, y_test, scaler_knn_26_path = knn_regression(df_26_path, df_sub, features_sub, 'Num_26_path')

0.9333333333333333
[[476  27]
 [  7   0]]


In [141]:
df_26_path_m = df_26_path_m.set_index(keys=['JOB_SEEKER_ID'])
df_26_path = df_26_path.set_index(keys=['JOB_SEEKER_ID'])


In [142]:

for index, row in df_26_path_m.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    
    # Apply the same scaler used during training
    X_test_scaled = scaler_knn_26_path.transform(X_test)
    
    df_26_path.loc[index, 'Num_26_path_exp'] = best_model.predict(X_test_scaled)

df_26_path


,Num_26_path,Den_26_path,Num_26_path_exp
JOB_SEEKER_ID,,,
9386301003,0.0,0.0,0
8653691003,0.0,1.0,0
7870290004,0.0,1.0,0
6498977509,0.0,0.0,0
6638331004,0.0,0.0,0
...,...,...,...
2836891309,0.0,0.0,0
8771045004,0.0,1.0,0
15325790,0.0,0.0,0


### KNN - 52 Week outcomes

In [146]:

df_52 = pd.read_csv("C:/Users/AManalo/star_ratings_new/df_52_new.csv")
df_52_full = df_52[["Job_Seeker_ID", "Num_52_full", "Den_52_full"]]
df_52_full = df_52_full.dropna()
df_52_path = df_52[["Job_Seeker_ID", "Num_52_path", "Den_52_path"]]
df_52_path = df_52_path.dropna()


df_52_full = df_52_full.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_52_full['Num_52_full_exp'] = 0
# merge with df_merge_52
df_52_full_m = df_sub.merge(df_52_full, on=['JOB_SEEKER_ID'], how='inner')

df_52_full_m = df_52_full_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO", "WAGE_SUBSIDY_ELIGIBLE"]]
df_52_full_m = df_52_full_m.set_index(keys=['JOB_SEEKER_ID'])

df_52_path = df_52_path.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

df_52_path['Num_52_path_exp'] = 0
# merge with df_merge_52
df_52_path_m = df_sub.merge(df_52_path, on=['JOB_SEEKER_ID'], how='inner')

df_52_path_m = df_52_path_m[["JOB_SEEKER_ID", "CURRENT_CAPACITY_HOURS", "GENDER", "TIME_IN_SITE", "DISABILITY_TYPE", "AGE", "INDIGENOUS_IND", "IND_METRO", "WAGE_SUBSIDY_ELIGIBLE"]]
df_52_path_m = df_52_path_m.set_index(keys=['JOB_SEEKER_ID'])


#### 52 full exp

In [147]:
best_model, X_train, X_test, y_resampled, y_test, scaler_knn_full = knn_regression(df_52_full, df_sub, features_sub, 'Num_52_full')

0.6745098039215687
[[320 143]
 [ 23  24]]


In [148]:
df_52_full = df_52_full.set_index(keys=['JOB_SEEKER_ID'])

for index, row in df_52_full_m.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    
    # Apply the same scaler used during training
    X_test_scaled = scaler_knn_full.transform(X_test)
    
    df_52_full.loc[index, 'Num_52_full_exp'] = best_model.predict(X_test_scaled)

df_52_full


,Num_52_full,Den_52_full,Num_52_full_exp
JOB_SEEKER_ID,,,
9386301003,0.0,0.0,0
8653691003,0.0,1.0,0
7870290004,0.0,1.0,0
6498977509,1.0,1.0,1
6638331004,0.0,0.0,0
...,...,...,...
2836891309,0.0,0.0,1
8771045004,0.0,1.0,0
15325790,0.0,0.0,0


#### 52 path exp

In [149]:
best_model, X_train, X_test, y_resampled, y_test, scaler_knn_path = knn_regression(df_52_path, df_sub, features_sub, 'Num_52_path')

0.9862745098039216
[[503   7]
 [  0   0]]


In [150]:
df_52_path = df_52_path.set_index(keys=['JOB_SEEKER_ID'])
for index, row in df_52_path_m.iterrows():
    X_test = pd.Series(row[['CURRENT_CAPACITY_HOURS', 'DISABILITY_TYPE', 'AGE', 'WAGE_SUBSIDY_ELIGIBLE']])
    X_test = X_test.values.reshape(1, -1)
     # Apply the same scaler used during training
    
    # Apply the same scaler used during training
    X_test_scaled = scaler_knn_path.transform(X_test)
    
    df_52_path.loc[index, 'Num_52_path_exp'] = best_model.predict(X_test_scaled)

df_52_path

,Num_52_path,Den_52_path,Num_52_path_exp
JOB_SEEKER_ID,,,
9386301003,0.0,0.0,0
8653691003,0.0,0.0,0
7870290004,0.0,0.0,0
6498977509,0.0,0.0,0
6638331004,0.0,0.0,0
...,...,...,...
2836891309,0.0,0.0,0
8771045004,0.0,0.0,0
15325790,0.0,0.0,0


### Expected Performance using Stat Regression Model
- No. of participants (NOT the no. of times they denominate) in the DEN post-quarterisation per performance measure 
- Expected = no. of outcomes (participants in the numerator)
- Per contract per performance measure 


Method: 
1. Group data by jobseeker -> site -> contract 
- Bring in an additional dataframe that contains site/contract info.
- Merge using job_seeker_id to df_merge
- Aggregate into a groupBy object
2. Create a separate test set without labels ('1') for numerator 


#### Aggregating into a groupBy object
- by site -> contract


In [151]:
# reading in the site/contract dataframe 
df_scg  = pd.read_csv("C:/Users/AManalo/star_ratings_new/site_contract_groups.csv")
print(len(df_scg))
df_scg.dtypes

# pre-processing the dataframe
df_scg.isna().sum()

16996


Program                          0
ess_identifier                   0
Contract ID                    220
Site Code                      126
Site Description               126
ESA Code                         0
ESA Name                       126
State                          126
specialist_site_type_code    11683
dtype: int64

In [ ]:
# renaming the columns
df_scg = df_scg.rename(columns={'ESA Code' : 'esa_code'})

1. Site Code
2. Site Description 
can both be null -> as these both generate unique rows in the government released star ratings file. For the sake of data type conversion -> Empty values are encoded as 'XXXX' 

NOT NULL - defines each group (keys of the group)
1. Contract ID
2. ESA Code
3. Speciality_site_type_code


In [ ]:
df_scg[df_scg['Contract ID'] == '0212822B'].to_csv('missing_test.csv')

In [152]:
df_scg = df_scg[df_scg['Contract ID'].isna() == False]
df_scg = df_scg[df_scg['specialist_site_type_code'].isna() == False]

In [ ]:
df_scg['Site Code'] = df_scg['Site Code'].fillna(value='XXXX')
df_scg

In [153]:
df_scg = df_scg.rename(columns={'ess_identifier' : 'JOB_SEEKER_ID', 'Site Code' : 'SITE_CODE', 'specialist_site_type_code' : 'SPECIALIST_SITE_TYPE_CODE', 'Site Description' : 'SITE_DESCRIPTION', 'Contract ID' : 'CONTRACT_ID' })
df_scg['JOB_SEEKER_ID'] = df_scg['JOB_SEEKER_ID'].astype('int64')


Joining to df_merge
- RIGHT join as the grouping needs to exist


#### checking the number of unique job seeker ids
as stored in each performance measure dataframe 


In [ ]:
print('df_13 no. of Job Seekers: ' + str(len(df_13)))
print('df_26 no. of Job Seekers: ' + str(len(df_26)))
print('df_52 no. of Job Seekers: ' + str(len(df_52)))

In [ ]:
df_13.columns
df_26 = df_26.rename(columns={'Job_Seeker_ID' :'JOB_SEEKER_ID'})


In [ ]:
df_52.columns
df_52 = df_52.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

In [ ]:
df_26_full

### Merging pre-quarterisation dataframes:

In [154]:
df_13_full = df_13_full.reset_index()
df_26_full = df_26_full.reset_index()
df_26_full = df_26_full.reset_index()

In [155]:
df_26_path = df_26_path.reset_index()
df_52_path = df_52_path.reset_index()


In [167]:
df_26_wrkast = df_26_wrkast.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})

In [168]:
df_merge1 = df_scg.merge(df_13_full, on=['JOB_SEEKER_ID'], how='left')
df_merge2 = df_merge1.merge(df_26_full, on=['JOB_SEEKER_ID'], how='left')
df_merge3 = df_merge2.merge(df_26_path, on=['JOB_SEEKER_ID'], how='left')


In [169]:
df_merge3

,Program,JOB_SEEKER_ID,CONTRACT_ID,SITE_CODE,SITE_DESCRIPTION,ESA Code,ESA Name,State,SPECIALIST_SITE_TYPE_CODE,Num_13,Den_13,Num_13_full_exp,index,Num_26_full,Den_26_full,Num_26_full_exp,Num_26_path,Den_26_path,Num_26_path_exp
0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,1.0,1.0,0.0,328.0,1.0,1.0,0.0,0.0,0.0,0.0
1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,0.0,1.0,1.0,1798.0,1.0,1.0,1.0,0.0,0.0,0.0
2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,1.0,1.0,0.0,637.0,1.0,1.0,0.0,0.0,0.0,0.0
3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,1.0,1.0,0.0,3240.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5308,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5309,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5310,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5311,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Job_Seeker_ID,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp
0,9386301003,0.0,0.0,0
1,8653691003,0.0,0.0,0
2,7870290004,0.0,0.0,0
3,6498977509,0.0,0.0,0
4,6638331004,0.0,0.0,0
...,...,...,...,...
4965,2836891309,0.0,0.0,0
4966,8771045004,0.0,0.0,0
4967,15325790,0.0,0.0,0
4968,690092007,0.0,0.0,0


In [170]:
df_merge4 = df_merge3.merge(df_26_wrkast, on=['JOB_SEEKER_ID'], how='left')
df_merge5 = df_merge4.merge(df_52_full, on=['JOB_SEEKER_ID'], how='left')
df_merge6 = df_merge5.merge(df_52_path, on=['JOB_SEEKER_ID'], how='left')
df_merge_final = df_merge6


print('length of df_merge_final: ' + str(len(df_merge_final)))

length of df_merge_final: 5313


In [171]:
df_merge_final = df_merge_final.dropna(how='any',thresh=7, axis=0) # thresh set to 7 (at least 1 Numerator/Denominator value not na)
df_merge_final.to_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')

In [172]:
# bringing in apv_stream_participant_versions 

df_apv = pd.read_csv('C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/apv_stream_participant_versions.csv')
# joining to df_merge
df_apv = df_apv.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
df_comb2 = df_apv.merge(df_scg, on=['JOB_SEEKER_ID'], how='inner')



### Merging post-quarterisation dataframes:

Subsetting the post quarterisation dataframes to their subtypes: 

In [173]:
df_post_13 = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_13_post.csv')
df_post_26 = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_post_26.csv')
df_post_52 = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_post_52.csv')

df_post_13 = df_post_13.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
df_post_13 = df_post_13.drop(columns=['Unnamed: 0'])

df_post_26 = df_post_26.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
df_post_26 = df_post_26.drop(columns=['Unnamed: 0'])

df_post_52 = df_post_52.rename(columns={'Job_Seeker_ID' : 'JOB_SEEKER_ID'})
df_post_52 = df_post_52.drop(columns=['Unnamed: 0'])


In [ ]:
df_post_52

Merging the dataframes

In [ ]:
df_post_13

In [ ]:
df_scg

Merging to site_contract_level - joining on job-seeker-id

In [ ]:
df_scg

In [ ]:
df_post_26 

Renaming the columns for df_scg (site-level data) -> rename so that it is lowercare

In [ ]:
df_scg = df_scg.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_DESCRIPTION' : 'Site_Name', 'SPECIALIST_SITE_TYPE_CODE' : 'specialist_site_type_code'})
df_scg = df_scg.drop(columns=['Program', 'SITE_CODE', 'ESA_CODE', 'ESA Name', 'State'])

In [ ]:
df_scg

In [ ]:
df_post_26

In [ ]:
df_post_13

### Merge the dataframes together to site level 

In [174]:


# creating merged data frame -> to pass to star_ratings_new_results
df_merge1 = df_post_13.merge(df_post_26, on=['JOB_SEEKER_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code'], how='inner')
df_merge2 = df_merge1.merge(df_post_52, on=['JOB_SEEKER_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code'], how='inner')

df_merge_final = df_merge2


In [ ]:
df_post_52

In [175]:

df_merge_final.to_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')

Expected performance vs Denominator
Denominator = no. of participants in caseload under each contract
1. Creating a data frame to store all denominator/expected performance
2. Populate the dataframe and calculated expected performance rate % 
Aggregate using groupBy attribute features

In [ ]:
#df_exp_per = pd.DataFrame(columns=['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE', 'DEN_13_FULL', 'EXPECTED_13', 'EXPECTED_PER_RATE_13', 'DENOMINATOR_26', 'EXPECTED_26', 'EXPECTED_PER_RATE_26', 'DENOMINATOR_52', 'EXPECTED_PER_RATE_52', 'EXPECTED_PER_RATE_52'])

len(df_merge_final)                       

In [ ]:
df_merge_final

### Expected performance for each job seeker ID
- Then sum the expected performance -> contract level
- Model coefficient and intercept will differ based on the input (training/test set)
- Each equation is uniquely constructed based on the get_eqn function

In [ ]:
# calculated above
df_13_full['Num_13_full_exp'].value_counts()


In [ ]:
df_26_full['Num_26_full_exp'].value_counts()

In [ ]:
df_26_path['Num_26_path_exp'].value_counts()

In [ ]:
df_52_full['Num_52_full_exp'].value_counts()